In [16]:
import pandas as pd

from src.spacy_models.spacy_pipe_build import SpacyPipeBuild
from src.spacy_models.spacy_pipe_process import SpacyPipeProcess
from src.spacy_models.spacy_utils import SpacyExt, SpacyTask, ExtractionType
from src.settings.enums import NaturalLanguage
from src.data.data_loader import DataLoader

In [17]:
nat_lang = NaturalLanguage.DE

In [18]:
coref = SpacyPipeProcess(natural_language=nat_lang, spacy_task=SpacyTask.COREF, use_gpu=True, extraction_type=ExtractionType.TRADITIONAL)

GPU is used: True
custom extensions "init_extensions" initialized
regex_entity_pattern for own_regex_search were compiled.
Function "own_regex_search" initialized
Function "own_coref_resolve" initialized


In [19]:
coref.nlp.pipe_names

['tok2vec', 'senter', 'own_regex_search', 'xx_coref_resolve']

In [20]:
# ner.nlp.disabled

In [21]:
# ner.nlp.analyze_pipes()

In [22]:
dl = DataLoader()
year = 2023
month = 5
df = dl.load_monthly_df(year=year, month=month)

In [122]:
# without findings: 10, 11
index = 44
isin = df.loc[index, 'art_isin']
company = df.loc[index, 'art_company_name']
print('ISIN:', isin, '---', 'COMP:', company)
print('-----------------------------------')
text = df.loc[index, 'pp_art_text']
text

ISIN: GB0007980591 --- COMP: BP PLC
-----------------------------------


'Der Oelkonzern BP hat im ersten Quartal die niedrigeren Oel- und Gaspreise zu spueren bekommen. Der operative Gewinn schrumpfte aber nicht so stark wie befuerchtet. Dank des weiterhin hohen Geldzuflusses kuendigte BP am Dienstag in London auch den Rueckkauf weiterer Aktien an - aber mit gedrosseltem Tempo. Unter diesem Eindruck verlor der Aktienkurs im fruehen Londoner Handel mehr als 5 Prozent. Im ersten Quartal ging der um Sondereffekte bereinigte Gewinn im Jahresvergleich um rund ein Fuenftel auf knapp fuenf Milliarden US-Dollar zurueck, wie der Konzern mitteilte. Damit uebertraf der Konzern die Erwartungen der Experten. Inklusive Sondereffekten betrug der Gewinn 8,2 Milliarden Dollar. Wegen der Abschreibung seiner Beteiligung an dem russischen Oelkonzern Rosneft und des Rueckzugs aus dem Geschaeft in Russland war im Vorjahreszeitraum unter dem Strich ein Verlust in Hoehe von 20,4 Milliarden Dollar angefallen. Unterdessen verlangsamt BP das Tempo bei den Aktienrueckkaeufen. Auch we

In [123]:
%%time
doc = coref.nlp(text=text)

##### ->>>>>>>>>> COREF: PipeFunc.CUST_EXT_VAL_WAS_SET: True
cluster_dict: {'head': (0, 17, 'Der Oelkonzern BP'), 1: (214, 216, 'BP'), 2: (551, 562, 'der Konzern'), 3: (590, 601, 'der Konzern'), 4: (721, 727, 'seiner'), 5: (951, 953, 'BP'), 6: (1014, 1037, 'der grossen Oelkonzerne'), 7: (1139, 1154, 'Die Unternehmen'), 8: (1224, 1226, 'BP'), 9: (1227, 1232, 'seine'), 10: (1404, 1419, 'das Unternehmen'), 11: (1489, 1491, 'es'), 12: (1556, 1567, 'der Konzern'), 13: (1568, 1573, 'seine'), 14: (1724, 1739, 'das Unternehmen'), 15: (1823, 1825, 'es')}
cluster_dict: {'head': (22, 39, 'im ersten Quartal'), 1: (399, 416, 'Im ersten Quartal')}
cluster_dict: {'head': (25, 39, 'ersten Quartal'), 1: (402, 416, 'ersten Quartal')}
cluster_dict: {'head': (417, 421, 'ging'), 1: (574, 579, 'Damit')}
cluster_dict: {'head': (422, 460, 'der um Sondereffekte bereinigte Gewinn'), 1: (664, 674, 'der Gewinn')}
cluster_dict: {'head': (1002, 1037, 'die Gewinne der grossen Oelkonzerne'), 1: (1092, 1095, 'sie')}
c

In [124]:
for ent in doc.ents:
    print(ent)

BP
BP
der Konzern
der Konzern
seiner
BP
der grossen Oelkonzerne
Die Unternehmen
BP
seine
das Unternehmen
es
der Konzern
seine
das Unternehmen
es


In [125]:
for ent in doc.ents:
    print('ent.text:', ent.text, 
        # 'POS:', [t.pos_ for t in ent],
        '--- comp_name:', getattr(ent._, SpacyExt.COMP_NAME.value) ,
        '--- Symbol:', getattr(ent._, SpacyExt.COMP_SYMBOL.value), 
        '--- Set_in:', getattr(ent._, SpacyExt.SET_IN.value))

ent.text: BP --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: own_regex_search
ent.text: BP --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: own_regex_search
ent.text: der Konzern --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: xx_coref_resolve
ent.text: der Konzern --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: xx_coref_resolve
ent.text: seiner --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: xx_coref_resolve
ent.text: BP --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: own_regex_search
ent.text: der grossen Oelkonzerne --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: xx_coref_resolve
ent.text: Die Unternehmen --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: xx_coref_resolve
ent.text: BP --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: own_regex_search
ent.text: seine --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: xx_coref_resolve
ent.text: das Unternehmen --- comp_name: BP p.l.c. --- Symbol: BP.L --- Set_in: xx_coref_resolve
ent.

In [126]:
from spacy import displacy
options = { "colors": {"ORG": "red","ENT-RULER":"green", "OWN-REGEX":"orange", "ORG-PART":"blue", "FUZZY":"pink", "PER":"grey", "LOC":"gray", "XX-COREF":"blue"}}
displacy.render(doc, style='ent', options=options)

In [127]:
for sent in doc.sents:
    print(sent)
    print('----------------')

Der Oelkonzern BP hat im ersten Quartal die niedrigeren Oel- und Gaspreise zu spueren bekommen.
----------------
Der operative Gewinn schrumpfte aber nicht so stark wie befuerchtet.
----------------
Dank des weiterhin hohen Geldzuflusses kuendigte BP am Dienstag in London auch den Rueckkauf weiterer Aktien an - aber mit gedrosseltem Tempo.
----------------
Unter diesem Eindruck verlor der Aktienkurs im fruehen Londoner Handel mehr als 5 Prozent.
----------------
Im ersten Quartal ging der um Sondereffekte bereinigte Gewinn im Jahresvergleich um rund ein Fuenftel auf knapp fuenf Milliarden US-Dollar zurueck, wie der Konzern mitteilte.
----------------
Damit uebertraf der Konzern die Erwartungen der Experten.
----------------
Inklusive Sondereffekten betrug der Gewinn 8,2 Milliarden Dollar.
----------------
Wegen der Abschreibung seiner Beteiligung an dem russischen Oelkonzern Rosneft und des Rueckzugs aus dem Geschaeft in Russland war im Vorjahreszeitraum unter dem Strich ein Verlust in

In [128]:
coref.set_custom_tokenizer()

In [129]:
for sent in doc.sents:
    print(sent)
    print('----------------')

Der Oelkonzern BP hat im ersten Quartal die niedrigeren Oel- und Gaspreise zu spueren bekommen.
----------------
Der operative Gewinn schrumpfte aber nicht so stark wie befuerchtet.
----------------
Dank des weiterhin hohen Geldzuflusses kuendigte BP am Dienstag in London auch den Rueckkauf weiterer Aktien an - aber mit gedrosseltem Tempo.
----------------
Unter diesem Eindruck verlor der Aktienkurs im fruehen Londoner Handel mehr als 5 Prozent.
----------------
Im ersten Quartal ging der um Sondereffekte bereinigte Gewinn im Jahresvergleich um rund ein Fuenftel auf knapp fuenf Milliarden US-Dollar zurueck, wie der Konzern mitteilte.
----------------
Damit uebertraf der Konzern die Erwartungen der Experten.
----------------
Inklusive Sondereffekten betrug der Gewinn 8,2 Milliarden Dollar.
----------------
Wegen der Abschreibung seiner Beteiligung an dem russischen Oelkonzern Rosneft und des Rueckzugs aus dem Geschaeft in Russland war im Vorjahreszeitraum unter dem Strich ein Verlust in